In [ ]:
from new_thesis_library import *

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from multiprocessing import Pool

%matplotlib inline

# Previous day values

In [ ]:
moex_theta = np.array([4.836731, 0.08779514, -0.03113246, 0.00511053])

## L-BFGS-B

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    if i != 0:        
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'L-BFGS-B', theta0 = thetas[i - 1])        
    else:
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'L-BFGS-B', theta0 = moex_theta)

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/l_bfgs_b_recurrent.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/l_bfgs_b_recurrent.csv')

## Powell

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    if i != 0:        
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'Powell', theta0 = thetas[i - 1])        
    else:
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'Powell', theta0 = moex_theta)

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/powell_recurrent.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/powell_recurrent.csv')

## Nelder-Mead

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    if i != 0:        
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'nelder-mead', theta0 = thetas[i - 1])        
    else:
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'nelder-mead', theta0 = moex_theta)

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/nelder_mead_recurrent.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/nelder_mead_recurrent.csv')

## Trust-constr

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    if i != 0:        
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'trust-constr', theta0 = thetas[i - 1])        
    else:
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'trust-constr', theta0 = moex_theta)

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/trust_constr_recurrent.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/trust_constr_recurrent.csv')

# Random values

In [ ]:
rand_thetas = []

for i in range(16):
    rand_thetas.append(pd.read_csv(path + 'Data/New_data/Random_thetas/rand_' + str(i + 1) + '.csv', 
                                   index_col='TRADEDATE', parse_dates=True))

## L-BFGS-B

### 1 - 4

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_l_bfgs_b, rand_thetas[:4])

### 5 - 8

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_l_bfgs_b, rand_thetas[4:8])

### 9 - 12

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_l_bfgs_b, rand_thetas[8:12])

### 13 - 16

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_l_bfgs_b, rand_thetas[12:16])

## Powell

### 1 - 4

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_powell, rand_thetas[:4])

### 5 - 8

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_powell, rand_thetas[4:8])

### 9 - 12

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_powell, rand_thetas[8:12])

### 13 - 16

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_powell, rand_thetas[12:16])

## Nelder-Mead

### 1 - 4

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_nelder_mead, rand_thetas[:4])

### 5 - 8

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_nelder_mead, rand_thetas[4:8])

### 9 - 12

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_nelder_mead, rand_thetas[8:12])

### 13 - 16

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_nelder_mead, rand_thetas[12:16])

## Trust-constr

### 1 - 4

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_trust_constr, rand_thetas[:4])

### 5 - 8

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_trust_constr, rand_thetas[4:8])

### 9 - 12

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_trust_constr, rand_thetas[8:12])

### 13 - 16

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_trust_constr, rand_thetas[12:16])

## Leftovers after ValueError

In [ ]:
%%time

with Pool(processes=4) as pool: 

    pool.map(parallel_trust_constr, [rand_thetas[5], rand_thetas[7], rand_thetas[8], rand_thetas[10]])

In [ ]:
%%time

with Pool(processes=2) as pool: 

    pool.map(parallel_trust_constr, [rand_thetas[13], rand_thetas[15]])

# Diebold, Li weights

In [ ]:
diebold_li_values = pd.read_csv(path + 'Data/New_data/diebold_li.csv', 
                                index_col='TRADEDATE', parse_dates=True)

## L-BFGS-B

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'L-BFGS-B', theta0 = np.copy(diebold_li_values.iloc[i]))        

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/l_bfgs_b_diebold_li.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/l_bfgs_b_diebold_li.csv')

## Powell

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'Powell', theta0 = np.copy(diebold_li_values.iloc[i]))        

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/powell_diebold_li.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/powell_diebold_li.csv')

## Nelder-Mead

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'Nelder-Mead', theta0 = np.copy(diebold_li_values.iloc[i]))        

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/nelder_mead_diebold_li.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/nelder_mead_diebold_li.csv')

## Trust-constr

In [ ]:
%%time

thetas = np.zeros([len(dates), 4])
time = np.zeros(len(dates))

for i in range(len(dates)):
    try:
        thetas[i], time[i] = optimize_on_day_with_starting_values(i, 'trust-constr', theta0 = np.copy(diebold_li_values.iloc[i]))            
    except ValueError:
        print('ValueError on day', i)
        thetas[i] = np.nan
        time[i] = np.nan

thetas = pd.DataFrame(thetas, index = dates, columns = ['tau', 'beta0', 'beta1', 'beta2'])
thetas.to_csv(path + 'Thetas/trust_constr_diebold_li.csv')

time = pd.DataFrame(time, index = dates, columns = ['Seconds'])
time.to_csv(path + 'Time/trust_constr_diebold_li.csv')